## genTestCasesTool

### Install package

In [ ]:
%pip install -U langchain-google-genai
%pip install -U google-genai
%pip install --upgrade --quiet  langchain-google-genai
%pip install dotenv
%pip install langchain-ollama
%pip install --upgrade --quiet  langchain-ollama

!pip install langchain-community
!pip install unstructured
!pip install pdfminer
!pip install "unstructured[all-docs]" 

## Connect Ollama Local model

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI 
#from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load= load_dotenv('../.env')
 

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


tc_llm = ChatGoogleGenerativeAI( 
   model="gemini-2.0-flash"
)

## Create tools
### Tools for Parallel Execution 

In [3]:
from langchain.tools import tool
from langchain.prompts import PromptTemplate


import os
from langchain_community.document_loaders import (
    UnstructuredFileLoader
)


@tool
def generate_test_cases_from_pdf(pdf_path: str) -> str:
    "Reads the requirement from the PDF and generates atleast 10 test cases scenarios in  BDD style format"



    Docs = "Pdf"
    documents= []

    for file in os.listdir(Docs):
        filpath = os.path.join(Docs, file)
        print(filpath)
        loader = UnstructuredFileLoader(filpath)
        #documents.extend(loader.load())
        docs= loader.load()
        for doc in docs:
            #documents.append(f"{filename} :: {doc.page_content.strip}")
            documents.append(f"{file} :: {doc.page_content.strip()}")

    requirement_txt = "\n\n" .join(documents[:3])[:1000]
  

    prompt_template = PromptTemplate.from_template(
        """
        You are a QA Automation Engineer. 
        Your task is to convert the following user story into at least 10 test cases.in Gherkin BDD style  format.
        Include combinations of valid, invalid, and edge cases & alternative scenarios.

        User Story: {requirement_txt}    

        Format:
        Feature: 
        Scenario:
            Given 
            When 
            Then
        And 



        """
                          

    )
    # ✅ Correct way to pass input
    prompt = prompt_template.invoke({"requirement_txt": requirement_txt})
    return llm.invoke(prompt) 

## initialize_agent

In [4]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[generate_test_cases_from_pdf],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,    
    verbose=True
)

result = agent.run("requirements.pdf")
print(result)

/tmp/ipykernel_121001/60982014.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/tmp/ipykernel_121001/60982014.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run("requirements.pdf")




> Entering new AgentExecutor chain...

Invoking: `generate_test_cases_from_pdf` with `{'pdf_path': 'requirements.pdf'}`


Pdf/LLM_introduction.pdf


/tmp/ipykernel_121001/1731837613.py:23: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(filpath)


content='As a QA Automation Engineer, I\'ve interpreted the provided text, which describes a research paper, as a "user story" for an educational platform or documentation system that hosts and delivers this "Self-contained Introduction to Large Language Models." The goal is to ensure this document effectively serves its intended audience (beginners with basic ML knowledge) and meets its described characteristics (self-contained, modularized).\n\nHere are 12 test cases in Gherkin BDD style:\n\n---\n\n**Feature: Learning about Large Language Models (LLMs) through the Self-contained Introduction**\n\n**Scenario 1: Successful Comprehension by Target Audience (Valid Case)**\n    Given a user with basic machine learning knowledge\n    When they access and read the "Self-contained Introduction to Large Language Models"\n    Then they should be able to understand the fundamental concepts of LLMs.\n\n**Scenario 2: Navigating the Modularized Design (Valid Case)**\n    Given a user is viewing th